# Data

In [ ]:
from google.colab import files, drive
import pandas as pd
import re
from tqdm.auto import tqdm
import json


drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('./drive/My Drive/work/qa_data.csv', sep='\t', encoding='utf-8')

In [ ]:
with open('./drive/My Drive/d/SeqDataSetAllData_TestDataExcluded_Flair_train.json', 'r', encoding='utf-8') as f:
  js_frame = json.load(f)

prefix for few-shot learning:

In [ ]:
#inds = [0, 150, 317, 321, 499, 3, 539, 561, 562] # for t5-large experiment
inds = [150, 321, 3, 0] 

full_text=''
for i in inds:
  text = f"""What are the objects and the aspect of comparison in the sentence '{df.iloc[i,0]}'
Objects: '{df.iloc[i,1]}', '{df.iloc[i,2]}'
Aspect: {df.iloc[i,3]}"""
  full_text+=text+'\n\n'


## split to sentence, obj1, obj2, aspect

In [ ]:
with open('./drive/My Drive/SeqDataSetAllData_TestDataExcluded_Flair_train.tsv', encoding='utf-8') as f:
  train = [sent.split('\n') for sent in f.read().split('\n\n')]
  
del train[-1]

In [ ]:
sentences = []
objects1 = []
objects2 = []
aspects = []

for sent in tqdm(train[:500], total=len(train[:500])):
  obj1 = []
  obj2 = []
  asp = []
  txt = []
  for word in sent:
    w = word.split('\t')
    txt.append(w[0])
    if w[1] == 'OBJ-1':
      obj1.append(w[0])
    elif w[1] == 'OBJ-2':
      obj2.append(w[0])
    elif w[1] == 'SHARED':
      obj1.append(w[0])
      obj2.append(w[0])
    elif w[1] == 'ASPECT':
      asp.append(w[0])
  object1 = ' '.join(obj1)
  object2 = ' '.join(obj2)
  sent_text = ' '.join(txt)
  aspect = ' '.join(asp)
  
  sentences.append(sent_text)
  objects1.append(object1)
  objects2.append(object2)
  aspects.append(aspect)
    


  0%|          | 0/500 [00:00<?, ?it/s]

## csv

In [ ]:
df = pd.DataFrame({'sentence':sentences, 'obj-1':objects1, 'obj-2':objects2, 'aspect':aspects})

In [ ]:
df.loc[df['aspect'] == '', 'aspect'] = '-'

In [ ]:
df.to_csv('./drive/My Drive/qa_data.csv', sep='\t', encoding='utf-8', index=False)

## json

In [ ]:
json_data = []
for i in range(len(sentences)):
  instruction = f"What are the objects and the aspect of comparison in the sentence '{sentences[i]}'?"
  input = ''
  output = f"There are two objects: {objects1[i]} and {objects2[i]}, and the aspect: {aspects[i]}."

  json_data.append({'instruction':instruction, 'input':'', 'output':output})

In [ ]:
js_frame = json.dumps(json_data)

In [ ]:
with open('./drive/My Drive/SeqDataSetAllData_TestDataExcluded_Flair_train.json', 'w', encoding='utf-8') as f:
  f.write(js_frame)

# LLaMA

In [ ]:
!pip install bitsandbytes
!pip install -q datasets loralib sentencepiece
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip install -q git+https://github.com/huggingface/peft.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.7 MB/s eta 0:00:00
  Installing b

In [ ]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig

tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")
model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)

In [ ]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

In [ ]:
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

def evaluate(instruction, input=None):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        return output.split("### Response:")[1].strip()

In [ ]:
answers = []
for i in tqdm(range(len(js_frame))):
  sent = js_frame[i]
  answer = evaluate(sent['instruction']).split('\n### Instruction:')[0]
  answer = re.sub('\n', ' ', answer)
  answers.append(answer.strip())

In [ ]:
df['llama_zero'] = answers

In [ ]:
df.to_csv('./drive/My Drive/llama_zeroshot.csv', sep='\t', encoding='utf-8', index=False)

# Dolly 

In [ ]:
%pip install "accelerate>=0.16.0,<1" "transformers[torch]>=4.28.1,<5" "torch>=1.13.1,<2" -q


In [ ]:
import torch
from transformers import pipeline

generate_text = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto") 


In [ ]:
answers = []
for i in tqdm(range(len(js_frame))): 
  sent = js_frame[i]
  answer = generate_text(sent['instruction'])[0]['generated_text']
  answers.append(answer)


In [ ]:
df['dolly_zero'] = answers

In [ ]:
df.to_csv('./drive/My Drive/dolly_zeroshot.csv', sep='\t', index=False, encoding='utf-8')

In [ ]:
answers = []
for i in tqdm(range(500)): 
  sent = js_frame[i]
  answer = generate_text(full_text+sent['instruction'])[0]['generated_text']
  answers.append(answer)

In [ ]:
df['dolly_few'] = answers

In [ ]:
df.to_csv('./drive/My Drive/dolly_fewshot.csv', sep='\t', index=False, encoding='utf-8')

# NeoGPT

In [ ]:
!pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.6 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline
generator = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B', max_length=250)


In [ ]:
answers = []
for i in tqdm(range(js_frame)): 
  sent = js_frame[i]
  answer = generator(sent['instruction'])[0]['generated_text']
  answers.append(answer)

In [ ]:
df['neogpt_zero'] = answers

In [ ]:
df.to_csv('./drive/My Drive/neogpt_zeroshot.csv', sep='\t', index=False, encoding='utf-8')

In [ ]:
answers = []
for i in tqdm(range(500)): 
  sent = js_frame[i]
  answer = generator(full_text+sent['instruction'])[0]['generated_text']
  answers.append(answer)

In [ ]:
df['neogpt_few'] = answers

df.to_csv('./drive/My Drive/neogpt_fewshot.csv', sep='\t', index=False, encoding='utf-8')

# GPT-J

In [ ]:
!pip install transformers -q
!pip install accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 8.2 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B") 
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")



In [ ]:
def generate_text(inp_text):
  input_ids = tokenizer(inp_text, return_tensors="pt").input_ids
  gen_tokens = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.9,
    max_length=300, # zero - 150
  )
  gen_text = tokenizer.batch_decode(gen_tokens)[0]
  return gen_text

In [ ]:
answers = []
for i in tqdm(range(len(js_frame))): 
  sent = js_frame[i]
  answer = generate_text(sent['instruction'])
  answers.append(answer)


In [ ]:
df['gptj_zero'] = answers
df.to_csv('./drive/My Drive/gptj_zeroshot.csv', sep='\t', encoding='utf-8', index=False)

In [ ]:
answers = []
for i in tqdm(range(500)): 
  sent = js_frame[i]
  answer = generate_text(full_text+sent['instruction'])
  answers.append(answer)


In [ ]:
df['gptj_few'] = answers
df.to_csv('./drive/My Drive/gptj_fewshot.csv', sep='\t', encoding='utf-8', index=False)

# GPT-2

In [ ]:
!pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.4 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel 
import torch

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [ ]:
from transformers import pipeline

text_generation = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [ ]:
answers = []
for i in tqdm(range(js_frame)): 
  sent = js_frame[i]
  answer = text_generation(sent['instruction'], max_length=300, do_sample=False)[0]['generated_text']
  answers.append(answer)

In [ ]:
df['gpt2_zero'] = answers

In [ ]:
df.to_csv('./drive/My Drive/gpt2_zeroshot.csv', sep='\t', encoding='utf-8', index=False)

In [ ]:
answers = []
for i in tqdm(range(500)): 
  sent = js_frame[i]
  answer = text_generation(full_text+sent['instruction'], max_length=300, do_sample=False)[0]['generated_text']
  answers.append(answer)


In [ ]:
df['gpt2_few'] = answers

In [ ]:
df.to_csv('./drive/My Drive/gpt2_fewshot.csv', sep='\t', encoding='utf-8', index=False)

# T5


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 101.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("t5-large")
model = T5ForConditionalGeneration.from_pretrained("t5-large")

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
def generator(inp_text):
  input_ids = tokenizer(inp_text, return_tensors="pt").input_ids

  outputs = model.generate(input_ids, max_length=100)
  preds = [
    tokenizer.decode(out, skip_special_tokens=True)
    for out in outputs
    ]
  return preds[0]

In [ ]:
answers = []
for i in tqdm(range(js_frame)):
  sent = js_frame[i]
  answer = generator(sent['instruction'])
  answers.append(answer)

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
df['t5_zero'] = answers

In [ ]:
df.to_csv('./drive/My Drive/t5_zeroshot.csv', sep='\t', encoding='utf-8', index=False)

In [ ]:
answers = []
for i in tqdm(range(500)): 
  sent = js_frame[i]
  answer = generator(full_text+sent['instruction'])
  answers.append(answer)

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
df['t5_few'] = answers

In [ ]:
df.to_csv('./drive/My Drive/t5_fewshot.csv', sep='\t', encoding='utf-8', index=False)

# FLAN-T5

In [ ]:
!pip install transformers -q

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large") 


In [ ]:
answers = []
for i in tqdm(range(len(js_frame))):
  sent = js_frame[i]
  inputs = tokenizer(sent['instruction'], return_tensors="pt") 
  outputs = model.generate(**inputs, max_length=100) 
  answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
  answers.append(answer)

In [ ]:
df['flant5_zero'] = answers

In [ ]:
df.to_csv('./drive/My Drive/flant5_zeroshot.csv', sep='\t', encoding='utf-8', index=False)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xl")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl") # xl 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
answers = []
for i in tqdm(range(500)):
  sent = js_frame[i]
  inputs = tokenizer(full_text+sent['instruction'], return_tensors="pt") 
  outputs = model.generate(**inputs, max_length=100) 
  answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
  answers.append(answer)

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
df['flant5_few'] = answers

In [ ]:
df.to_csv('./drive/My Drive/flant5xl_fewshot.csv', sep='\t', encoding='utf-8', index=False)